# Tranform the Github Data  

#### Read the data

In [59]:
# import the libraries
import pandas as pd
import numpy as np
import sys

In [60]:
# read the data
df = pd.read_csv("../data/total_processed.csv", index_col=0)

In [61]:
df.shape

(601078, 55)

### Mean of the commit label

In [62]:
def mean(items):
    index = 0
    result = []
    value = 0

    for item in items:
        index = index + 1

        if item == 1:
            value = value + 1

        result.append((value/index)*100)

    return result

In [63]:
# create a dictionary project_name + mean_values for the commit classification
projects = df['name'].unique()
commit_mean_dict = {}

for project in projects:
    commits = df[df.name==project].results
    mean_commits = mean(commits)
    commit_mean_dict[project] = mean_commits

In [64]:
# replace all the data
df['mean_commits'] = 0

for project in projects:
    df.loc[df['name'] == project, 'mean_commits'] = commit_mean_dict[project]

### Variance of the column results

In [65]:
for project in projects:
    df.loc[df['name'] == project, 'variance_commits'] = df[df['name']==project][['results', 'mean_commits']].var(ddof=0, axis=1)

### Set the phases of the project

In [72]:
def phases(entire_data):
    len_project = len(entire_data)
    phases = []
    
    end_phase_1 = len_project / 3
    end_phase_1 = int(end_phase_1)
    end_phase_2 = end_phase_1 + end_phase_1
    end_phase_2 = int(end_phase_2)

    i = 0

    for item in entire_data:
        if i < end_phase_1:
            phases.append(1)
            i = i + 1
        elif i >= end_phase_1 and i < end_phase_2:
            phases.append(2)
            i = i + 1
        else:
            phases.append(3)
            i = i + 1
        
    return phases

In [73]:
# create a dictionary project_name + phases for the commit classification
commit_phases_dict = {}

for project in projects:
    commits = df[df.name==project].results
    phases_commits = phases(commits)
    commit_phases_dict[project] = phases_commits

In [74]:
# replace all the data
df['phases_project'] = 0

for project in projects:
    df.loc[df['name'] == project, 'phases_project'] = commit_phases_dict[project]

In [75]:
df[df['name']=='stat-cookbook']

,sha,message,comment_count,date,total_deletions,total_additions,total,comments,additions,deletions,...,language_Shell,language_TypeScript,total_files,results,dateonly,isweekend,isnight,mean_commits,variance_commits,phases_project
337,ccc5c3febf1e8e91ef39eee75f931ba3d9cb3714,Initial import.,0.0,2011-03-05T04:48:15Z,0.0,25084,25084,[],"[10, 11, 49, 2519, 258, 851, 738, 4241, 1741, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0.0,0.0,20,0.0,Saturday,1.0,1.0,0.000000,0.000000,1.0
315,ffcd1a8e8f6404d9c294c3d7931874cf70365f9b,Restyle table with booktabs package.\n\nSugges...,0.0,2011-03-05T05:40:40Z,32.0,34,66,[],[34],[32],...,0.0,0.0,1,0.0,Saturday,1.0,1.0,0.000000,0.000000,1.0
305,7184d955144f7aff75015353b0c999a8463f0430,Add Student's t and F distribution.\n\nSuggest...,0.0,2011-03-05T06:02:25Z,2.0,18,20,[],[18],[2],...,0.0,0.0,1,0.0,Saturday,1.0,0.0,0.000000,0.000000,1.0
304,2025c1ca462fb3244ab57754954eddcedca0b95a,Move Gamma and Beta function to Math section.,0.0,2011-03-05T06:24:05Z,54.0,47,101,[],[47],[54],...,0.0,0.0,1,0.0,Saturday,1.0,0.0,0.000000,0.000000,1.0
303,cc0d9761095b9349d1abdaec422078f62a81dfab,Use datetime package for date formatting.\n\nA...,0.0,2011-03-05T06:53:47Z,3.0,4,7,[],[4],[3],...,0.0,0.0,1,0.0,Saturday,1.0,0.0,0.000000,0.000000,1.0
291,dd58a67c1dff3f58ca7371d0d2a4aa94ba36827c,Fix headings for finite and binomial series.,0.0,2011-03-05T09:01:46Z,9.0,6,15,[],[6],[9],...,0.0,0.0,1,1.0,Saturday,1.0,0.0,16.666667,61.361111,1.0
290,7d42c17c2acf785b54265c61c2644fc35e159859,Update .gitignore.,0.0,2011-03-05T09:02:44Z,0.0,2,2,[],[2],[0],...,0.0,0.0,1,0.0,Saturday,1.0,0.0,14.285714,51.020408,1.0
277,002a24bc3015b5299f19a01db9953c6864a6d80b,Remove day from cover page date.,0.0,2011-03-05T09:06:25Z,1.0,1,2,[],[1],[1],...,0.0,0.0,1,0.0,Saturday,1.0,0.0,12.500000,39.062500,1.0
276,a48389d3b8029be49db0c8f18ff449281cba76d8,Add R graphs for Student's t and F distribution.,0.0,2011-03-05T19:35:07Z,0.0,7185,7185,[],"[2, 33, 2889, 4261]","[0, 0, 0, 0]",...,0.0,0.0,4,0.0,Saturday,1.0,0.0,11.111111,30.864198,1.0
268,607b99cb2ca59cac497adece7313da311c978993,Tweak distribution graphs.\n\n- Fix a typo in ...,0.0,2011-03-06T15:55:24Z,25684.0,25673,51357,[],"[3, 588, 435, 4036, 1561, 2609, 1096, 165, 208...","[1, 590, 432, 4040, 1561, 2609, 1096, 175, 208...",...,0.0,0.0,17,1.0,Sunday,1.0,0.0,20.000000,90.250000,1.0
